In [2]:
import numpy as np
import pandas as pd
import math
import re
import scipy.stats as sps
from tqdm import tqdm
from datetime import datetime, date
from scipy.sparse import hstack, vstack
from scipy.special import logit, expit

from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.linear_model import LogisticRegression, LinearRegression

import warnings
warnings.filterwarnings("ignore")

In [3]:
!ls data

players.pkl  results.pkl  tournaments.pkl  working


In [4]:
DIR = "data/"

### 1. Data reading and analysing:

In [5]:
%%time
players = pd.read_pickle(DIR + "players.pkl") 
results = pd.read_pickle(DIR + "results.pkl")
tournaments = pd.read_pickle(DIR + "tournaments.pkl")

CPU times: user 13.4 s, sys: 3.19 s, total: 16.6 s
Wall time: 16.6 s


In [6]:
# split tournaments to train/test
tournaments_df = pd.DataFrame.from_dict(
    tournaments, orient="index", columns=["id", "name", "dateStart"]
)
tournaments_df["dateStart"] = pd.to_datetime(tournaments_df["dateStart"]).apply(lambda x: x.date())
train_tournaments_df = tournaments_df[
    (tournaments_df["dateStart"] >= date(2019, 1, 1))
    & (tournaments_df["dateStart"] <= date(2019, 12, 31))
]
test_tournaments_df = tournaments_df[
    (tournaments_df["dateStart"] >= date(2020, 1, 1))
    & (tournaments_df["dateStart"] <= date(2020, 12, 31))
]

In [7]:
# now, we need to filter data with results
def filter(tour_id: int):
    result = results[tour_id]
    masks = []
    team_ids = []
    team_names = []
    team_position = []
    player_ids = []
    player_names = []
    for team in result:
        questionsTotal = team.get("questionsTotal")
        if questionsTotal is None:
            continue
        if questionsTotal <= 0:
            continue
        position = team.get("position")
        if position is None:
            continue
        masks.append(team.get("mask").replace("X", "0").replace("?", "0"))
        team_ids.append(team["team"]["id"])
        team_names.append(team["team"]["name"])
        team_position.append(position)
        player_ids.append([player["player"]["id"] for player in team["teamMembers"]])
        player_names.append(
            [
                "{} {} {}".format(
                    player["player"]["surname"],
                    player["player"]["name"],
                    player["player"]["patronymic"],
                )
                for player in team["teamMembers"]
            ]
        )

    return masks, team_ids, team_names, team_position, player_ids, player_names


def update_dataframe(dataframe: pd.DataFrame):
    (
        dataframe["mask"],
        dataframe["team_id"],
        dataframe["team_name"],
        dataframe["team_position"],
        dataframe["player_id"],
        dataframe["player_name"],
    ) = zip(*dataframe["id"].apply(filter))
    dataframe = (
        dataframe.explode(
            [
                "mask",
                "team_id",
                "team_name",
                "team_position",
                "player_id",
                "player_name",
            ]
        )
        .explode(["player_id", "player_name"])
        .reset_index(drop=True)
    )
    dataframe.dropna(inplace=True)
    dataframe["mask_len"] = dataframe["mask"].apply(len)
    dataframe = dataframe.merge(
        dataframe.groupby("id")["mask_len"].max().rename("max_len"),
        left_on="id",
        right_index=True,
    )
    dataframe = dataframe[dataframe["mask_len"] == dataframe["max_len"]].reset_index(
        drop=True
    )
    dataframe = dataframe[
        [
            "id",
            "name",
            "team_id",
            "team_name",
            "team_position",
            "player_id",
            "player_name",
            "mask",
            "mask_len",
        ]
    ]
    return dataframe

In [8]:
%%time
train_tournaments_df2 = update_dataframe(train_tournaments_df)
test_tournaments_df2 = update_dataframe(test_tournaments_df)
train_tournaments_df2.to_csv("data/working/train_tournaments_df2.csv", index=False)
test_tournaments_df2.to_csv("data/working/test_tournaments_df2.csv", index=False)
# train_tournaments_df2 = pd.read_csv("data/working/train_tournaments_df2.csv")
# test_tournaments_df2 = pd.read_csv("data/working/test_tournaments_df2.csv")

CPU times: user 1.42 s, sys: 98.3 ms, total: 1.52 s
Wall time: 1.52 s


#### Questions dataframe:
Now, we need to create raw data about player/question stats (After all we add question rating to their indexes)

In [9]:
def questions_dataframe(dataframe: pd.DataFrame) -> pd.DataFrame:
    tmp_df = dataframe[['id', 'mask', 'mask_len', 'player_id']]
    tmp_df['points'] = tmp_df['mask'].apply(lambda x: [int(i) for i in x])
    
    tmp_df['question_id'] = tmp_df.apply(lambda x: [(x.id * 10 ** 3 + i) for i in range(x.mask_len)], axis=1)
    tmp_df2 = tmp_df.explode(['points', 'question_id'])
    return tmp_df2

In [10]:
%%time
train_questions_raw = questions_dataframe(train_tournaments_df2)
test_questions_raw = questions_dataframe(test_tournaments_df2)
train_questions_raw.to_csv("data/working/train_questions_raw.csv", index=False)
test_questions_raw.to_csv("data/working/test_questions_raw.csv", index=False)
# train_questions_raw = pd.read_csv("data/working/train_questions_raw.csv")
# test_questions_raw = pd.read_csv("data/working/test_questions_raw.csv")

CPU times: user 15 s, sys: 2.29 s, total: 17.3 s
Wall time: 17.3 s


#### Question complexity:
As question complexity we can use share of correct answers:

In [11]:
def complexity_estimation(dataframe: pd.DataFrame) -> pd.DataFrame:
    tmp_df = dataframe[['player_id', 'points', 'question_id']]
    tmp_df['correct'] = tmp_df.groupby('question_id')['points'].transform('sum')
    tmp_df['total'] = tmp_df.groupby('question_id')['points'].transform('count')
    tmp_df['complexity'] = tmp_df['correct'] / tmp_df['total']  
    
    return tmp_df[['player_id', 'question_id', 'complexity', 'points']]

In [12]:
%%time
train_questions = complexity_estimation(train_questions_raw)
test_questions = complexity_estimation(test_questions_raw)
train_questions.to_csv("data/working/train_questions.csv", index=False)
test_questions.to_csv("data/working/test_questions.csv", index=False)
# train_questions = pd.read_csv("data/working/train_questions.csv")
# test_questions = pd.read_csv("data/working/train_questions.csv")

CPU times: user 11.5 s, sys: 3.44 s, total: 14.9 s
Wall time: 14.9 s


### 2. Baseline model:
#### We can start to work with first baseline - analyse how player work with question depends on its complexity
About realisation - we can use personal LogisticRegression for each player or total, but player_id will be a part of features. Above, we create datasets to train and predict. 
1. Train - pairs player_id - complexity in train dataset;
2. Test - pairs player_id and range complexity to predict individual ability of player to work with tasks with different complexity.

In [13]:
X_train = train_questions[['player_id', 'complexity']].values
y_train = train_questions['points'].astype('int').values

In [14]:
%%time
players_model_df = pd.DataFrame(players.keys(), columns=['player_id'])
players_model_df['complexity'] = [np.arange(0, 1, 0.05).tolist()] * players_model_df.shape[0]
players_model_df = players_model_df.explode('complexity')
players_model_df = players_model_df[players_model_df['player_id'].isin(train_questions['player_id'])]
X_predict = players_model_df[['player_id', 'complexity']].values

CPU times: user 800 ms, sys: 276 ms, total: 1.08 s
Wall time: 1.07 s


In [15]:
players_list = players_model_df['player_id'].drop_duplicates().to_list()

Firstly, try to user player_id and complexity as features value:

In [16]:
players_model_df['rates1'] = LogisticRegression(random_state = 42).fit(X_train, y_train).predict_proba(X_predict)[:, 1]
players_model_df['rates1'] = players_model_df['rates1'].apply(lambda x: round(x, 3))

In [17]:
assert players_model_df.groupby(['player_id'])['rates1'].nunique().sum() == players_model_df['player_id'].nunique()

As we can see, way to user player_id in features dataset is not good, because final label depends only on player_id and fill same values to points. I think, as a variant use individual logistic regression for every person.

In [18]:
def train(players_list: list, train_questions: pd.DataFrame, players_model_df: pd.DataFrame) -> pd.DataFrame:
    for player_id in tqdm(players_list):
        X_tmp_train = train_questions.loc[
            train_questions["player_id"] == player_id, "complexity"
        ].values
        y_tmp_train = (
            train_questions.loc[train_questions["player_id"] == player_id, "points"]
            .astype("int")
            .values
        )
        X_tmp_predict = players_model_df.loc[
            players_model_df["player_id"] == player_id, "complexity"
        ].values
        players_model_df.loc[players_model_df["player_id"] == player_id, "rates2"] = (
            LogisticRegression(random_state=42)
            .fit(X_tmp_train.reshape(-1, 1), y_tmp_train)
            .predict_proba(X_tmp_predict.reshape(-1, 1))[:, 1]
        )
    return players_model_df

players_model_df = train(players_list, train_questions, players_model_df)

In [19]:
players_model_df.to_csv("data/working/players_model_df.csv", index=False)
#players_model_df = pd.read_csv("data/working/players_model_df.csv")

Final rate of player - mean rate for every task complexity. It's will decsribe how he will work will average task. Now, we can display list of top-players:

In [20]:
players_model_df2 = players_model_df.groupby("player_id")["rates2"].mean().reset_index()
players_model_df2["player_name"] = players_model_df2["player_id"].apply(
    lambda x: " ".join([i for i in list(players[x].values())[1:] if i is not None])
)
players_model_df2.sort_values(by="rates2", ascending=False).reset_index(drop=True).head(
    10
)

,player_id,rates2,player_name
0,121433,0.915940,София Романовна Савенко
1,186002,0.907916,Инга Андрисовна Лоренц
2,202410,0.903794,Валентина Подюкова
3,133504,0.892291,София Евгеньевна Лебедева
4,170977,0.890857,Давид Сергеевич Кан
5,171845,0.890857,Михаил Владимирович Завьялов
6,215496,0.884946,Наталья Артемьева
7,215497,0.884946,Екатерина Горелова
8,215495,0.884946,Юлия Крюкова
9,199963,0.874346,Елена Борисовна Бровченко


### Team ranging and correlation:
#### Team rate:
As team rate let's take probability of correct prediction by one player in team: $P(team=1) = 1 - \prod P(player=0)$.

In [21]:
team_rating_df = test_tournaments_df2.merge(
    players_model_df2, on="player_id", how="left"
)
team_rating_df["inv_rates2"] = 1 - team_rating_df["rates2"]
team_rating_df["team_rate"] = 1 - team_rating_df.groupby(
    ["id", "name", "team_id", "team_name", "team_position"]
)["inv_rates2"].transform("prod")
team_rating_df2 = team_rating_df[['id', 'team_id', 'team_position', 'team_rate']].drop_duplicates()
team_rating_df2['team_rate'] = - team_rating_df2['team_rate']

In [22]:
def estimate_correlation(team_rating_df: pd.DataFrame):
    spearman = 0
    kendall = 0
    tours_list = team_rating_df['id'].drop_duplicates().to_list()
    tours_num = len(tours_list)
    for tour_id in tours_list:
        tmp_df = team_rating_df[team_rating_df['id'] == tour_id]
        tmp_spearman = sps.spearmanr(tmp_df['team_rate'].to_list(), tmp_df['team_position'].to_list())[0]
        tmp_kendall = sps.kendalltau(tmp_df['team_rate'].to_list(), tmp_df['team_position'].to_list())[0]
        if not (np.isnan(tmp_spearman) or np.isnan(tmp_kendall)):
            spearman += tmp_spearman
            kendall += tmp_kendall
    return spearman / tours_num, kendall / tours_num

In [23]:
results = estimate_correlation(team_rating_df2)
print("Spearman correlation on test dataset: ", results[0])
print("Kendall correlation on test dataset: ", results[1])

Spearman correlation on test dataset:  0.6933628862304252
Kendall correlation on test dataset:  0.5396450864708623


### 4. EM-algorithm
As dataset we will use dummy-dataframe with players and questions. Hidden variable here - comparison between probability of player and probability of team. When we are using linear regression player's weights will be more if player take more contribution to final probability.

Create dataset for test:

In [24]:
test_dataset = team_rating_df[["id", "team_id", "team_position", "player_id"]].drop_duplicates()

And for train with hidden variable:

Create variable state_prob - with we will estimate in every iteration, it's base value got from counting team probabilities based on baseline players probabilities:

In [25]:
df1 = train_questions_raw[['id','player_id', 'points','question_id']].drop_duplicates()
df2 = train_tournaments_df2[['id', 'team_id', 'player_id']].drop_duplicates()
em_dataset = df1.merge(df2, on=['id', 'player_id'])

In [26]:
eps = 1e-5

state_prob = np.array(em_dataset['points'])
state_prob = np.clip(state_prob, eps, 1-eps)

Finally, we have train dataset...

In [27]:
em_df_X = em_dataset.copy()
em_df_X["player_id"] = em_df_X["player_id"].astype("category")
em_df_X["question_id"] = em_df_X["question_id"].astype("category")

em_X = pd.get_dummies(em_df_X[["player_id", "question_id"]], sparse = True)
em_dataset.head()

,id,player_id,points,question_id,team_id
0,4772,6212,1,4772000,45556
1,4772,6212,1,4772001,45556
2,4772,6212,1,4772002,45556
3,4772,6212,1,4772003,45556
4,4772,6212,1,4772004,45556


...Start probability...

In [28]:
state_prob

array([9.9999e-01, 9.9999e-01, 9.9999e-01, ..., 1.0000e-05, 1.0000e-05,
       1.0000e-05])

... dataset for evaluating.

In [29]:
test_dataset.head()

,id,team_id,team_position,player_id
0,4957,49804,1.0,30152
1,4957,49804,1.0,30270
2,4957,49804,1.0,27822
3,4957,49804,1.0,28751
4,4957,49804,1.0,27403


Create model:

In [30]:
em_model = LinearRegression(n_jobs = -1)

And start process:

In [31]:
for _ in tqdm(range(5)):

    em_model.fit(em_X, logit(state_prob))
    # test and count correlation
    em_players = [int(i.split("_")[-1]) for i in em_X.columns if "player_id" in i]
    em_coefs = em_model.coef_[: len(em_players)]
    em_probs = [1 / (1 + np.exp(-i)) for i in em_coefs]
    df_with_player_probs = pd.DataFrame(
        {"player_id": em_players, "coefs": em_coefs, "probs": em_probs}
    )
    test_dataset2 = (
        test_dataset[["id", "team_id", "team_position", "player_id"]]
        .drop_duplicates()
        .merge(df_with_player_probs, on="player_id")
    )
    test_dataset2["inv_probs"] = 1 - test_dataset2["probs"]
    test_dataset2["team_rate"] = 1 - (
        test_dataset2.groupby(["id", "team_id", "team_position"])["inv_probs"]
        .transform('prod')
    )
    test_dataset2 = test_dataset2[
        ["id", "team_id", "team_position", "team_rate"]
    ].drop_duplicates()
    test_dataset2["team_rate"] = -test_dataset2["team_rate"]

    results = estimate_correlation(test_dataset2)
    print("Spearman correlation on test dataset: ", results[0])
    print("Kendall correlation on test dataset: ", results[1])

    state_prob = expit(em_model.predict(em_X))
    em_dataset2 = em_dataset.copy()
    em_dataset2["prob"] = state_prob
    em_dataset2["inv_prob"] = 1 - em_dataset2["prob"]
    em_dataset2 = em_dataset2.merge(
        em_dataset2.groupby(["question_id", "team_id"])["inv_prob"].prod().rename("Z"),
        left_on=["question_id", "team_id"],
        right_index=True,
    )
    em_dataset2["Z"] = em_dataset2["prob"] / (1 - em_dataset2["Z"])
    em_dataset2["Z"] = em_dataset2["Z"] * em_dataset2["points"]
    state_prob = np.clip(em_dataset2["Z"], eps, 1 - eps)
    print("------------------------------------------------------")

  0%|          | 0/5 [00:00<?, ?it/s]

Spearman correlation on test dataset:  0.7592329624973166
Kendall correlation on test dataset:  0.6133494162117403


 20%|██        | 1/5 [12:21<49:26, 741.69s/it]

------------------------------------------------------
Spearman correlation on test dataset:  0.7770867644092331
Kendall correlation on test dataset:  0.622830311225666


 40%|████      | 2/5 [25:44<38:53, 777.85s/it]

...

------------------------------------------------------
Spearman correlation on test dataset:  0.7810814696268344
Kendall correlation on test dataset:  0.623784735049618


 80%|████████  | 4/5 [50:43<12:38, 758.41s/it]

------------------------------------------------------
Spearman correlation on test dataset:  0.7815176119137353
Kendall correlation on test dataset:  0.6240074003217699


100%|██████████| 5/5 [1:03:19<00:00, 759.85s/it]

------------------------------------------------------


#### Show top-10 tournaments with largest complexity:

In [65]:
em_questions = [int(i.split("_")[-1]) for i in em_X.columns if "question_id" in i]
em_question_coefs = em_model.coef_[-len(em_questions):]
em_question_probs = [1 / (1 + np.exp(-i)) for i in em_question_coefs]

In [66]:
questions_rating_df = pd.DataFrame(
    {
        "question_id": em_questions,
        "coefs": em_question_coefs,
        "probs": em_question_probs,
    }
)

In [67]:
questions_rating_df["tournament_id"] = questions_rating_df["question_id"].apply(
    lambda x: int(x / 1000)
)

In [68]:
top_10_df = (
    questions_rating_df.groupby("tournament_id")["probs"]
    .mean()
    .reset_index()
    .sort_values(by="probs", ascending=True)
    .reset_index(drop=True)
    .head(10)
)
top_10_df['name'] = top_10_df["tournament_id"].apply(lambda x: tournaments[x]['name'])

In [69]:
top_10_df

,tournament_id,probs,name
0,5928,0.026003,Угрюмый Ёрш
1,5684,0.037090,Синхрон высшей лиги Москвы
2,5159,0.042181,Первенство правого полушария
3,6101,0.044674,Воображаемый музей
4,5693,0.053202,Знание – Сила VI
5,5587,0.055778,Записки охотника
6,5083,0.059257,Ускользающая сова
7,5943,0.063342,Чемпионат Мира. Этап 2 Группа С
8,5942,0.064180,Чемпионат Мира. Этап 2. Группа В
9,5465,0.066711,Чемпионат России


#### Final thoughts:
1. Baseline model estimate ability of each player to solve questions with differents complexities;
2. EM-model correct idea and trained on hidden information about players, teams and questions;
3. EM-model improved correlation metrics;
4. At results we can see in top some tournaments from big cities and world stages. 